In [2]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv("/dlabdata1/lugeon/dmoz_domain_category.csv.gz")
df.shape

(2488259, 2)

In [16]:
df['cat_split'] = df.apply(lambda row: row.cat_labels_en.split('/'), axis=1)
df['cat_len'] = df.apply(lambda row: len(row.cat_split), axis=1)

In [17]:
def catk_or_none(row, k):
    if row.cat_len >= k:
        return row.cat_split[k-1]
    else:
        return None

In [18]:
df['main_cat'] = df.apply(lambda row: catk_or_none(row, 2), axis=1)
df.main_cat.unique()

array(['World', 'Computers', 'Business', 'Regional', 'Society', 'Arts',
       'Health', 'Science', 'Reference', 'Recreation', 'Shopping',
       'Sports', 'Games', 'News', 'Home', 'Regional|Top', 'News|Top'],
      dtype=object)

In [74]:
matchings = pd.read_csv("/dlabdata1/lugeon/categories.csv", sep=';')

In [75]:
matchings

,lang,id,Computers,Business,Society,Arts,Health,Science,Reference,Recreation,Shopping,Sports,Games,News,Home
0,English,en,Computers,Business,Society,Arts,Health,Science,Reference,Recreation,Shopping,Sports,Games,News,Home
1,Deutsch,de,Computer,economy,society,Culture,health,science,Knowledge,leisure,Online-Shops,Sport,games,media,At_home
2,Français,fr,data_processing,Business_and_Economy,Society,Arts,Health,Sciences,References,Hobbies,Online_Stores,Sports,Games,topicality,House
3,Russian,ru,computers,Business,Society,Art,Health,science,Information_sources,Leisure,Purchases,sport,Games,News_&amp;_Media,House
4,Español,es,Computing,Business,Society,Art,Health,Science_and_Technology,Reference/Education,Free_time,Shopping,sports,Games,Media,Home
5,Türkçe,tr,Computer,Economy_and_Business,Society,Arts_and_Entertainment,Health,Science,resources,0,Shopping,Spore,Games,Press_and_Publications,Home
6,Japanese,ja,Computer,business,society,Art,health,science,All_kinds_of_information,Recreation,online_shop,Sport,game,news,family
7,Português,pt,Computing,Business,Society,Art,Cheers,Science,Reference,Hobbies,Shopping,Sports,Games,News,Thigh
8,Chinese_Simplified,zh,computer,business,society,art,health,science,reference,Leisure,shopping,physical_education,game,news,family
9,Italiano,it,Computer,Business,Society,Art,Health,Science,Consultation,Free_time,Online_shopping,Sport,Games,News,Home


In [21]:
df_world = df[df.main_cat == 'World']
df_world['lang'] = df_world.apply(lambda row: catk_or_none(row, 3), axis=1)
df_world['lang_cat'] = df_world.apply(lambda row: catk_or_none(row, 4), axis=1)

<ipython-input-21-8dca46849ecf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_world['lang'] = df_world.apply(lambda row: catk_or_none(row, 3), axis=1)
<ipython-input-21-8dca46849ecf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_world['lang_cat'] = df_world.apply(lambda row: catk_or_none(row, 4), axis=1)


In [22]:
df_world.head()

,domain,cat_labels_en,cat_split,cat_len,main_cat,lang,lang_cat
0,sdcastroverde.com,Top/World/Galego/regional/Galicia/Lugo/municip...,"[Top, World, Galego, regional, Galicia, Lugo, ...",8,World,Galego,regional
2,zschachwitz-tischtennis.de,Top/World/Deutsch/Sport/ball_Sports/table_tenn...,"[Top, World, Deutsch, Sport, ball_Sports, tabl...",9,World,Deutsch,Sport
3,www.penguin.cz,Top/World/Česky/Computers/Software/Operating_s...,"[Top, World, Česky, Computers, Software, Opera...",8,World,Česky,Computers
6,medlawcenter.com.ua,Top/World/Ukrainian/Countries_and_regions/Euro...,"[Top, World, Ukrainian, Countries_and_regions,...",19,World,Ukrainian,Countries_and_regions
10,www.wasserstrahlen.com,Top/World/Deutsch/economy/Industrial_Goods_and...,"[Top, World, Deutsch, economy, Industrial_Good...",6,World,Deutsch,economy


In [78]:
std_cat = matchings.columns[2:]
std_cat

Index(['Computers', 'Business', 'Society', 'Arts', 'Health', 'Science',
       'Reference', 'Recreation', 'Shopping', 'Sports', 'Games', 'News',
       'Home'],
      dtype='object')

In [181]:
pd.options.mode.chained_assignment = None

df_main = pd.DataFrame(columns = ['domain', 'lang', 'lang_id', 'cat0'])

for index, row in matchings.iterrows():
    this_lang = row['lang']
    if this_lang != 'English':
        this_lang_id = row['id']
        df_this_lang = df_world[df_world.lang == this_lang]
        df_this_lang['lang_id'] = df_this_lang.apply(lambda x: this_lang_id, axis=1)
        for main_cat in std_cat:
            if row[main_cat] != '0':
                this_cat = row[main_cat].split('/')
                for cat in this_cat:
                    df_this_cat = df_this_lang[df_this_lang.lang_cat == cat]
                    df_this_cat['cat0'] = df_this_cat.apply(lambda x: main_cat, axis=1)
                    df_main = df_main.append(df_this_cat[['domain', 'lang', 'lang_id', 'cat0']])

In [182]:
df_main

,domain,lang,lang_id,cat0
184,www.delphipraxis.net,Deutsch,de,Computers
263,www.central-it.de,Deutsch,de,Computers
731,www.aus-wandern.de,Deutsch,de,Computers
1072,blog.cvjm-nuernberg.de,Deutsch,de,Computers
1101,www.ges-training.de,Deutsch,de,Computers
...,...,...,...,...
2466754,www.civuenne.com,Italiano,it,Home
2480198,www.rilibri.it,Italiano,it,Home
2481694,www.ricettegustose.it,Italiano,it,Home
2484416,www.casaborghi.com,Italiano,it,Home


In [183]:
index = df.main_cat.apply(lambda cat: cat in std_cat)
df_english = df[index]
df_english.head()

,domain,cat_labels_en,cat_split,cat_len,main_cat
1,www.232analyzer.com,Top/Computers/Hardware/Test_Equipment/Analyzers,"[Top, Computers, Hardware, Test_Equipment, Ana...",5,Computers
4,www.cs-interiors.co.uk,Top/Business/Construction_and_Maintenance/Mate...,"[Top, Business, Construction_and_Maintenance, ...",6,Business
7,ccc-stl.org,Top/Society/Religion_and_Spirituality/Christia...,"[Top, Society, Religion_and_Spirituality, Chri...",9,Society
11,www.utahwebdesign.com,Top/Computers/Internet/Web_Design_and_Developm...,"[Top, Computers, Internet, Web_Design_and_Deve...",7,Computers
13,www.hampsteadstage.org,Top/Arts/Performing_Arts/Theatre/Troupes_and_C...,"[Top, Arts, Performing_Arts, Theatre, Troupes_...",14,Arts


In [184]:
df_english = df_english[['domain', 'main_cat']]
df_english['lang'] = df_english.apply(lambda row: 'English', axis=1)
df_english['lang_id'] = df_english.apply(lambda row: 'en', axis=1)
df_english.rename(columns={'main_cat': 'cat0'}, inplace=True)
df_english.head()

,domain,cat0,lang,lang_id
1,www.232analyzer.com,Computers,English,en
4,www.cs-interiors.co.uk,Business,English,en
7,ccc-stl.org,Society,English,en
11,www.utahwebdesign.com,Computers,English,en
13,www.hampsteadstage.org,Arts,English,en


In [185]:
df_main = df_main.append(df_english)
df_main.rename(columns={'domain': 'url'}, inplace=True)

In [186]:
df_main

,url,lang,lang_id,cat0
184,www.delphipraxis.net,Deutsch,de,Computers
263,www.central-it.de,Deutsch,de,Computers
731,www.aus-wandern.de,Deutsch,de,Computers
1072,blog.cvjm-nuernberg.de,Deutsch,de,Computers
1101,www.ges-training.de,Deutsch,de,Computers
...,...,...,...,...
2488203,www.stompin-ground.com,English,en,Arts
2488226,www.quarrypark.co.uk,English,en,Sports
2488230,www.sugarcreekwines.com,English,en,Recreation
2488233,www.aera.net,English,en,Reference


In [191]:
df_main.reset_index(inplace=True)
df_main.rename(columns={'index': 'uid'}, inplace=True)
df_main

,uid,url,lang,lang_id,cat0
0,184,www.delphipraxis.net,Deutsch,de,Computers
1,263,www.central-it.de,Deutsch,de,Computers
2,731,www.aus-wandern.de,Deutsch,de,Computers
3,1072,blog.cvjm-nuernberg.de,Deutsch,de,Computers
4,1101,www.ges-training.de,Deutsch,de,Computers
...,...,...,...,...,...
1025176,2488203,www.stompin-ground.com,English,en,Arts
1025177,2488226,www.quarrypark.co.uk,English,en,Sports
1025178,2488230,www.sugarcreekwines.com,English,en,Recreation
1025179,2488233,www.aera.net,English,en,Reference


In [192]:
df_counts = pd.DataFrame([])

for l in matchings.lang:
    df_counts = df_counts.append(df_main[df_main.lang == l].cat0.value_counts())

df_counts.index = matchings.lang.values

In [193]:
df_counts['total'] = df_counts.apply(lambda row: row.sum(), axis=1)

In [195]:
cat_count = df_counts.apply(lambda col: col.sum(), axis=0)

In [196]:
df_counts.loc['total'] = cat_count

In [197]:
df_counts

,Arts,Business,Computers,Games,Health,Home,News,Recreation,Reference,Science,Shopping,Society,Sports,total
English,66721.0,148144.0,45194.0,10246.0,24218.0,6952.0,3696.0,46095.0,21663.0,28138.0,54062.0,82079.0,34890.0,572098.0
Deutsch,21157.0,44737.0,12373.0,3270.0,20049.0,1752.0,1684.0,17893.0,3810.0,6208.0,12681.0,19261.0,18036.0,182911.0
Français,11205.0,8934.0,4611.0,2009.0,4054.0,1628.0,531.0,6558.0,2731.0,4140.0,2470.0,7314.0,2086.0,58271.0
Russian,4922.0,12361.0,4176.0,1350.0,1895.0,1290.0,145.0,4172.0,806.0,1941.0,2441.0,5570.0,3279.0,44348.0
Español,3722.0,2105.0,1045.0,907.0,956.0,295.0,300.0,2369.0,2672.0,1219.0,560.0,3149.0,534.0,19833.0
Türkçe,2050.0,7588.0,2395.0,296.0,1419.0,204.0,476.0,NaN,2307.0,425.0,1394.0,2127.0,518.0,21199.0
Japanese,9471.0,28969.0,1175.0,1092.0,2556.0,867.0,954.0,2751.0,1202.0,1649.0,4095.0,5417.0,4302.0,64500.0
Português,736.0,823.0,415.0,80.0,238.0,56.0,32.0,217.0,308.0,286.0,87.0,744.0,93.0,4115.0
Chinese_Simplified,953.0,5173.0,1417.0,448.0,642.0,133.0,431.0,657.0,1994.0,741.0,403.0,1261.0,228.0,14481.0
Italiano,6323.0,17831.0,1303.0,1207.0,1428.0,751.0,367.0,2515.0,1173.0,1296.0,2642.0,4392.0,2197.0,43425.0


In [198]:
df_main.to_csv('/dlabdata1/lugeon/websites_dmoz_multi.gz', compression='gzip')